Pada modul sebelumnya kita sudah belajar tentang TFDS. Perlu kita ketahui bahwa tidak seluruh dataset yang kita butuhkan tersedia di TFDS sehingga, kita perlu tahu caranya untuk mengubah data tersebut ke dalam format TFDS. Nah, di modul ini, kita akan belajar bagaimana caranya mengubah sebuah dataset (berupa CSV atau Pandas DataFrame) menjadi format TFDS dengan cara yang efisien.

Dataset yang digunakan berasal dari kaggle dan dapat diunduh di [sini](https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv). Setiap baris dalam dataset ini mendeskripsikan berlian dengan detail spesifikasi pada setiap kolomnya. Kita akan menggunakan dataset ini untuk memprediksi harga berlian. 

Sebenarnya, harga dalam dataset ini berupa range alias rentang angka, namun untuk mempermudah proses pembelajaran kita akan membagi harga tersebut menjadi biner, yaitu rendah atau tinggi.

Berikut merupakan [deskripsi](https://www.kaggle.com/shivam2503/diamonds) setiap kolom dari dataset tersebut.

Nah, jadi bagaimana caranya untuk memproses dataset berupa CSV menjadi format TFDS?

1. Pertama Import Tensorflow dan library lainnya yang dibutuhkan:

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
import logging
logging.getLogger('tensorflow').disabled = True

2. Selanjutnya gunakan Pandas DataFrame untuk membaca file CSV

In [3]:
url = 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
df = pd.read_csv(url)
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


3. Setelah data telah dalam format dataframe, Split dataset menjadi training, test, dan validation

In [4]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


4. Kemudian bungkus dataframe dengan tf.data agar memungkinkan untuk menggunakan feature column sebagai jembatan dari dataframe menjadi fitur yang digunakan untuk train model. Jika file CSV nya sangat besar, kita harus menggunakan tf.data untuk membaca file tersebut dari harddisk.

In [5]:
#Membuat target dari prediksi model
# price 0 = low; 1 = high
df['target'] = np.where(df['price']==327, 0, 1)

# Drop un-used columns.
df = df.drop(columns=['price'])

In [6]:
# Cara untuk membuat dataset tf.data dari pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [7]:
batch_size = 10 #bath ukuran kecil untuk demonstrasi
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [8]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of carat:', feature_batch['carat'])
  print('A batch of targets:', label_batch )

Every feature: ['Unnamed: 0', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']
A batch of carat: tf.Tensor([0.32 1.54 0.3  0.57 0.31 0.98 0.57 1.03 1.55 1.02], shape=(10,), dtype=float64)
A batch of targets: tf.Tensor([1 1 1 1 1 1 1 1 1 1], shape=(10,), dtype=int64)


In [9]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())



---


Ada beberapa jenis dari feature column yang dapat menjadi masukan TensorFlow:
*   Numeric column: Numeric column merupakan jenis feature column yang paling sederhana dan digunakan untuk merepresentasikan sebuah fitur dengan value yang apa adanya. Masukkan ke TensorFlow harus berupa angka dan numeric column sudah merupakan angka sehingga tidak perlu diubah lagi.



In [10]:
#numeric column
carat = feature_column.numeric_column('carat')
demo(carat)

[[1.34]
 [0.41]
 [1.21]
 [1.07]
 [1.03]
 [0.7 ]
 [0.33]
 [2.11]
 [0.51]
 [0.54]]


* Bucketized column: Terkadang kita memiliki dataset numerik yang memiliki value beragam dengan range cukup jauh. Daripada memasukkan setiap value ke dalam numeric column, kita dapat menggunakan bucketized column untuk membagi value-value tersebut.

In [11]:
#bucketized column 
carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1, 2])
demo(carat_buckets)

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]


* Categorical column: Kita tidak dapat memasukkan value String ke dalam TensorFlow. Oleh karena itu, categorical column digunakan untuk mengubah value String pada dataset (pada dataset ini contohnya cut dan clarity) menjadi angka.

In [12]:
#categorical
color_type = feature_column.categorical_column_with_vocabulary_list(
      'color', ['E', 'I','J','D','H', 'G','F'])

color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)

[[0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0.]]


* Embedding column: Jika kita memiliki banyak value dalam satu categorical column, lalu jenis value tersebut dapat bertambah seiring dengan berjalannya waktu, data tersebut menjadi tidak cocok jika hanya direpresentasikan dengan nilai 0 atau 1 seperti categorical column. Embedding column merepresentasikan satu categorical column dengan nilai yang beragam.

Key point: hasil embedding column menjadi maksimal jika sebuah categorical column memiliki banyak jenis value.

In [13]:
#embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

[[-0.77178466 -0.12883733  0.4102609   0.7924034  -0.46237087 -0.01370215]
 [ 0.79941124  0.34997225  0.21355931 -0.11238474 -0.37079796 -0.81241673]
 [-0.6000797   0.22089694  0.22134754  0.17122604 -0.47438246  0.09445841]
 [-0.77178466 -0.12883733  0.4102609   0.7924034  -0.46237087 -0.01370215]
 [ 0.519104   -0.78927094  0.05984591  0.44974688  0.11024602  0.16508313]
 [ 0.79941124  0.34997225  0.21355931 -0.11238474 -0.37079796 -0.81241673]
 [ 0.03281794  0.06444904  0.15904194  0.21820255  0.13572532 -0.5439751 ]
 [-0.77178466 -0.12883733  0.4102609   0.7924034  -0.46237087 -0.01370215]
 [ 0.79941124  0.34997225  0.21355931 -0.11238474 -0.37079796 -0.81241673]
 [ 0.79941124  0.34997225  0.21355931 -0.11238474 -0.37079796 -0.81241673]]


* Hashed feature column: Hashed feature column merupakan cara alternatif untuk merepresentasikan categorical column yang memiliki banyak jenis value. Kita dapat menentukan jumlah hash_buckets jauh lebih sedikit dari jumlah kategori yang sebenarnya untuk menghemat tempat.

Key point: kerugian dari teknik ini adalah dapat terjadi collision yang mana kategori berbeda dipetakan pada bucket yang sama.

In [14]:
#hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket(
      'clarity', hash_bucket_size=5)
demo(feature_column.indicator_column(clarity_hashed))

[[0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


* Crossed feature column: Crossed feature column menggabungkan banyak feature column menjadi satu feature column. Crossed feature column membuat column feature baru yang memungkinkan model untuk mempelajari weight berbeda untuk setiap kombinasi dari column feature.

**Catatan**: crossed feature column tidak membuat table dari seluruh kemungkinan kombinasi feature column karena ukurannya bisa menjadi sangat besar. Sebagai gantinya, crossed feature column didukung oleh hashed feature column sehingga dapat mendefinisikan ukuran table tersebut.

In [15]:
#cross feature
#data yang di cross harus berupa string, categorical, atau bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type],
                                                hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]




---



5. Kemudian buat feature layer sebagai input ke dalam model tf.Keras.

In [16]:
#Pilih feature column mana yang akan digunakan
feature_columns = []

In [17]:
#embedding column
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=8)
feature_columns.append(clarity_embedding)

In [18]:
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))

In [19]:
#membuat feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [20]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

6. Terakhir tulislah kode untuk create, compile, dan train model.

In [21]:
#create, compile, and train the model
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])
 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
 
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
1686/1686 [==============================] - 23s 12ms/step - loss: 0.0017 - accuracy: 0.9996 - val_loss: 4.0467e-04 - val_accuracy: 1.0000
Epoch 2/10
1686/1686 [==============================] - 20s 12ms/step - loss: 4.1840e-04 - accuracy: 1.0000 - val_loss: 4.8698e-04 - val_accuracy: 1.0000
Epoch 3/10
1686/1686 [==============================] - 20s 12ms/step - loss: 4.8168e-04 - accuracy: 1.0000 - val_loss: 4.1769e-04 - val_accuracy: 1.0000
Epoch 4/10
1686/1686 [==============================] - 20s 12ms/step - loss: 4.1576e-04 - accuracy: 1.0000 - val_loss: 5.0268e-04 - val_accuracy: 1.0000
Epoch 5/10
1686/1686 [==============================] - 20s 12ms/step - loss: 3.8363e-04 - accuracy: 1.0000 - val_loss: 3.4366e-04 - val_accuracy: 1.0000
Epoch 6/10
1686/1686 [==============================] - 19s 11ms/step - loss: 4.3851e-04 - accuracy: 1.0000 - val_loss: 3.4460e-04 - val_accuracy: 1.0000
Epoch 7/10
1686/1686 [==============================] - 20s 12ms/step - loss: 3.

In [22]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1686/1686 [==============================] - 8s 5ms/step - loss: 2.9724e-04 - accuracy: 1.0000
Accuracy 0.9999814629554749
